In [1]:
%matplotlib inline
#导入相关依赖库
import  os
import numpy as np
from matplotlib import pyplot as plt

import mindspore as ms
#context模块用于设置实验环境和实验设备
import mindspore.context as context
#dataset模块用于处理数据形成数据集
import mindspore.dataset as ds
#c_transforms模块用于转换数据类型
import mindspore.dataset.transforms.c_transforms as C
#vision.c_transforms模块用于转换图像，这是一个基于opencv的高级API
import mindspore.dataset.vision.c_transforms as CV
#导入Accuracy作为评价指标
from mindspore.nn.metrics import Accuracy
#nn中有各种神经网络层如：Dense，ReLu
from mindspore import nn
#Model用于创建模型对象，完成网络搭建和编译，并用于训练和评估
from mindspore.train import Model
#LossMonitor可以在训练过程中返回LOSS值作为监控指标
from mindspore.train.callback import  LossMonitor
#设定运行模式为动态图模式，并且运行设备为昇腾芯片
context.set_context(mode=context.GRAPH_MODE, device_target='CPU') 
from mindspore.train.callback import ModelCheckpoint,CheckpointConfig
from mindspore import load_checkpoint, load_param_into_net

In [2]:
DATA_DIR_TRAIN = "D:/DL/MNIST/MNIST/train" # 训练集信息
DATA_DIR_TEST = "D:/DL/MNIST/MNIST/test" # 测试集信息

In [3]:
def create_dataset(training=True, batch_size=128, resize=(28, 28),rescale=1/255, shift=-0.5, buffer_size=64):
    ds = ms.dataset.MnistDataset(DATA_DIR_TRAIN if training else DATA_DIR_TEST)
    
    #定义改变形状、归一化和更改图片维度的操作。
    #改为（28,28）的形状
    resize_op = CV.Resize(resize)
    #rescale方法可以对数据集进行归一化和标准化操作，这里就是将像素值归一到0和1之间，shift参数可以让值域偏移至-0.5和0.5之间
    rescale_op = CV.Rescale(rescale, shift)
    #由高度、宽度、深度改为深度、高度、宽度
    hwc2chw_op = CV.HWC2CHW()
    
    # 利用map操作对原数据集进行调整
    ds = ds.map(input_columns="image", operations=[resize_op, rescale_op, hwc2chw_op])
    ds = ds.map(input_columns="label", operations=C.TypeCast(ms.int32))
    #设定洗牌缓冲区的大小，从一定程度上控制打乱操作的混乱程度
    ds = ds.shuffle(buffer_size=buffer_size)
    #设定数据集的batch_size大小，并丢弃剩余的样本
    ds = ds.batch(batch_size, drop_remainder=True)
    
    return ds

In [4]:
#生成训练集
ds_train = create_dataset(True, batch_size=32)
#生成验证集，验证机不需要训练，所以不需要repeat
ds_eval = create_dataset(False, batch_size=32)

In [6]:
class soft_max(nn.Cell):      
    def __init__(self):
        super(soft_max, self).__init__()
        self.flatten = nn.Flatten()
        self.Linear = nn.Dense(784,10)
    
    def construct(self, input_x):
        output = self.flatten(input_x)
        output = self.Linear(output)
        return output 

In [8]:
lr = 0.001
num_epoch = 10
momentum = 0.9
net = soft_max()
loss = nn.loss.SoftmaxCrossEntropyWithLogits( sparse=True, reduction='mean')
metrics={"Accuracy": Accuracy()}
#定义优化器为Adam优化器，并设定学习率
opt = nn.Adam(net.trainable_params(), lr)

In [9]:
model = Model(net, loss, opt, metrics)

In [10]:
config_ck = CheckpointConfig(save_checkpoint_steps=1875, keep_checkpoint_max=10)
ckpt_cb = ModelCheckpoint(prefix='ForwardNN', directory='D:/DL/MNIST/Mindspore/model',config=config_ck)

In [11]:
model.train(num_epoch, ds_train,callbacks=[LossMonitor(1875), ckpt_cb],dataset_sink_mode=True)

[WARNING] ME(12604:19792,MainProcess):2022-03-22-13:13:09.700.011 [mindspore\train\model.py:550] The CPU cannot support dataset sink mode currently.So the training process will be performed with dataset not sink.


epoch: 1 step: 1875, loss is 0.17323295772075653
epoch: 2 step: 1875, loss is 0.09796681255102158
epoch: 3 step: 1875, loss is 0.3955184519290924
epoch: 4 step: 1875, loss is 0.4463110864162445
epoch: 5 step: 1875, loss is 0.32775571942329407
epoch: 6 step: 1875, loss is 0.23661170899868011
epoch: 7 step: 1875, loss is 0.24752239882946014
epoch: 8 step: 1875, loss is 0.16208776831626892
epoch: 9 step: 1875, loss is 0.3200516402721405
epoch: 10 step: 1875, loss is 0.289153516292572


In [12]:
#使用测试集评估模型，打印总体准确率
metrics_result=model.eval(ds_eval)
print(metrics_result)

[WARNING] ME(12604:19792,MainProcess):2022-03-22-13:13:43.603.40 [mindspore\train\model.py:968] CPU cannot support dataset sink mode currently.So the evaluating process will be performed with dataset non-sink mode.


{'Accuracy': 0.9197716346153846}


In [9]:
net_test = soft_max()
load_checkpoint('D:/DL/MNIST/Mindspore/model/ForwardNN_1-8_1875.ckpt',net = net_test)
Copymetrics = {
    'accuracy': nn.Accuracy(),
    'loss': nn.Loss(),
    'precision': nn.Precision(),
    'recall': nn.Recall(),
    'f1_score': nn.F1()
}
model_test = Model(net_test, loss,metrics=Copymetrics)
acc = model_test.eval(ds_eval)
acc

[WARNING] ME(8844:7500,MainProcess):2022-03-22-13:29:57.355.218 [mindspore\common\parameter.py:338] The parameter definition is deprecated.
Please set a unique name for the parameter 'Parameter (name=Parameter, shape=(10, 784), dtype=Float32, requires_grad=True)'.
[WARNING] ME(8844:7500,MainProcess):2022-03-22-13:29:57.356.216 [mindspore\common\parameter.py:338] The parameter definition is deprecated.
Please set a unique name for the parameter 'Parameter (name=Parameter, shape=(10,), dtype=Float32, requires_grad=True)'.
[WARNING] ME(8844:7500,MainProcess):2022-03-22-13:29:57.726.116 [mindspore\train\model.py:968] CPU cannot support dataset sink mode currently.So the evaluating process will be performed with dataset non-sink mode.


{'accuracy': 0.91796875,
 'loss': 0.29136004130570936,
 'precision': array([0.96236012, 0.96173913, 0.92323232, 0.88610039, 0.89278752,
        0.88154897, 0.96612022, 0.93465347, 0.8467433 , 0.92331933]),
 'recall': array([0.96728016, 0.97530864, 0.88651794, 0.91071429, 0.93660532,
        0.86966292, 0.92275574, 0.91828794, 0.90946502, 0.87288977]),
 'f1_score': array([0.96481387, 0.96847636, 0.90450272, 0.89823875, 0.91417166,
        0.87556561, 0.9439402 , 0.92639843, 0.87698413, 0.89739663])}